In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
df = pd.read_csv("C:/Users/pkitt/Downloads/playground-series-s5e6/train.csv")

In [4]:
df.cat =df.select_dtypes(include=[object]) 

C:\Users\pkitt\AppData\Local\Temp\ipykernel_7440\1653857449.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.cat =df.select_dtypes(include=[object])


In [5]:
df.num=df.select_dtypes(include=[np.number])

C:\Users\pkitt\AppData\Local\Temp\ipykernel_7440\610118065.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.num=df.select_dtypes(include=[np.number])


In [6]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,Clayey,Sugarcane,28-28
1,Sandy,Millets,28-28
2,Sandy,Millets,17-17-17
3,Sandy,Barley,10-26-26
4,Red,Paddy,DAP


In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
le = LabelEncoder()

In [9]:
df.cat["Soil Type"] = le.fit_transform(df.cat["Soil Type"])

In [10]:
df.cat["Crop Type"] = le.fit_transform(df.cat["Crop Type"])

In [11]:
df.cat["Fertilizer Name"] = le.fit_transform(df.cat["Fertilizer Name"])

In [12]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,1,8,4
1,4,4,4
2,4,4,2
3,4,0,0
4,3,6,5


In [13]:
df_train = pd.concat([df.num,df.cat],axis=1)

In [14]:
import tensorflow as tf

In [15]:
from tensorflow import keras

In [16]:
X = df_train[["Humidity","Moisture","Soil Type","Crop Type"]]

In [17]:
y = df_train[["Fertilizer Name"]]

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [56]:
import torch.nn as nn

class FertilizerMLP(nn.Module):
    def __init__(self, input_dim=4, hidden_dim=128, output_dim=7):
        super(FertilizerMLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.net(x)

In [70]:
import torch
import torch.nn as nn
import torch.optim as optim
model = FertilizerMLP()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.long).squeeze()

for epoch in range(1):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch} | Loss: {loss.item():.4f}")

Epoch 0 | Loss: 8.9182


In [64]:
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score

def evaluate_model(model, X_test, y_test):
    X_tensor = torch.tensor(X_test.to_numpy(), dtype=torch.float32)
    y_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.long).squeeze()

    model.eval()
    with torch.no_grad():
        outputs = model(X_tensor)
        predicted = torch.argmax(outputs, dim=1)

    acc = accuracy_score(y_tensor.numpy(), predicted.numpy())
    loss = nn.CrossEntropyLoss()(outputs, y_tensor).item()

    print(f"✅ Accuracy: {acc:.4f} | 📉 Loss: {loss:.4f}")

In [65]:
evaluate_model(model, X_test, y_test)

✅ Accuracy: 0.1514 | 📉 Loss: 2.9977


In [66]:
print(np.unique(y, return_counts=True))

(array([0, 1, 2, 3, 4, 5, 6]), array([113887, 114436, 112453, 110889, 111158,  94860,  92317],
      dtype=int64))
